In [46]:
import random
import gurobipy as gp
from gurobipy import GRB
import copy
import numpy as np

n=5
def generate_data(n):
    # Initializing an empty adjacency matrix with zeros
    W = [[0] * n for _ in range(n)]

    # Generate random edge weights (assuming non-negative weights)
    for i in range(n):
        for j in range(i+1, n):
            t = random.randint(0,2)
            if t == 0:
                weight = 0
            else:
                weight = random.randint(1,10)
            # weight = random.uniform(0, 2)  # Adjust the range as needed
            # Ensure that edges are undirected by setting W[i][j] = W[j][i] = weight
            W[i][j] = W[j][i] = weight
    return W

In [47]:
def BP(W,n):
    # Create a new model
    m = gp.Model("mip1")
    # Create variable

    x = {}
    x = m.addVars(range(n),vtype = "B", name="x")
    y = m.addVars(range(n),range(n),vtype = "B", name="y")

    m.setObjective(sum(W[i][j]*y[i,j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)
    m.addConstrs(y[i,j]<=(x[i]+x[j]) for i in range(n) for j in range(n))
    m.addConstrs(y[i,j]+x[i]+x[j]<=2 for i in range(n) for j in range(n))
    m.optimize()
    y_result=[]

    for v in x.values():
        if v.X == 0 or v.X==-0:
            y_result.append(0)
        else:
            y_result.append(1)
    return y_result

In [48]:
def count_weight(W:list, A:list, B:list):
    total_weight = 0
    for i in A:
        for j in B:
            total_weight += W[i][j]
    return total_weight

In [49]:
def GH(W, A, B, n):
    # print("The (sub)optimal set A is {}".format(A))
    # print("The (sub)optimal set B is {}".format(B))
    total_weight = count_weight(W, A, B)
    # print("The total weight of these sets is: {}".format(total_weight))

    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        # print("the old set A is: {}".format(A))
        # print("the old set B is: {}".format(B))
        if i in A:
            # print(str(r)+" is in A")
            temp_B.append(i)
            temp_A.remove(i)
        else:
            # print(str(r)+" is in B")
            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        # print("the new weight is: {}".format(new_weight))
        # print("the old weight is: {}".format(total_weight))
        # print("the new set A is: {}".format(temp_A))
        # print("the new set B is: {}".format(temp_B))
        if new_weight >= total_weight:
            # print('success this new set is an improvement')
            A = temp_A
            B = temp_B
            total_weight = new_weight
        # print()
    A = sorted(A)
    B = sorted(B)
    # print("The optimal set A from GH is {}".format(A))
    # print("The optimal set B from GH is {}".format(B))
    total_weight = count_weight(W, A, B)
    # print("The total weight of these sets is: {}".format(total_weight))
    return total_weight, A, B

In [50]:
def weight_change(W, A, B, current_weight):
    weight_change_list = []
    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        # print("the old set A is: {}".format(A))
        # print("the old set B is: {}".format(B))
        if i in A:
            # print(str(r)+" is in A")
            temp_B.append(i)
            temp_A.remove(i)
        else:
            # print(str(r)+" is in B")
            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        result = new_weight - current_weight
        weight_change_list.append(result)
    return weight_change_list

In [51]:
def random_set():
    A = []
    B = []
    xdata_gh = []

    for i in range(n):
        r = random.randint(0,1)
        if r == 0:
            xdata_gh.append(0)
            A.append(i)
        else:
            xdata_gh.append(1)
            B.append(i)
    return xdata_gh, A, B


In [52]:
def sum_weight(W):
    sum_W = []
    for i in W:
        sum_W.append(sum(i))
    return sum_W

In [53]:
def NN_data(k,n):
    xdata = []
    gh_weight_list = []
    GH_A_outcomes = []
    GH_B_outcomes = []
    ydata_list = []

    W = generate_data(n)
    ydata = BP(W,n)
    sum_W = sum_weight(W)

    temp_W = copy.deepcopy(W)
    for i in range(len(W)):
        temp_W[i].append(sum_W[i])

    for j in range(k):
        xdata_gh, A, B = random_set()
        
        ydata_list.append(ydata)

        weight = count_weight(W,A,B)
        weight_diff = weight_change(W,A,B,weight)

        temp_W1 = copy.deepcopy(temp_W)

        for v in range(len(W)):
            temp_W1[v].append(xdata_gh[v])
            temp_W1[v].append(weight_diff[v])
        xdata.append(temp_W1)

        gh_weight, A, B = GH(W, A, B, n)
        gh_weight_list.append(gh_weight)
        GH_A_outcomes.append(A)
        GH_B_outcomes.append(B)
        
    return xdata, ydata_list, gh_weight_list, GH_A_outcomes, GH_B_outcomes

In [54]:
k=200
xtrain_list, ytrain_list, gh_weight_list_train, GH_A_train, GH_B_train = NN_data(k,n)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 30 columns and 140 nonzeros
Model fingerprint: 0xa6fba993
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 36 rows and 18 columns
Presolve time: 0.00s
Presolved: 14 rows, 12 columns, 42 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 68.0000000

Root relaxation: objective 7.400000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

In [55]:
k=100
xtest_list, ytest_list, gh_weight_list_test, GH_A_test, GH_B_test = NN_data(k,n)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 30 columns and 140 nonzeros
Model fingerprint: 0x9dd00d37
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 38 rows and 19 columns
Presolve time: 0.00s
Presolved: 12 rows, 11 columns, 36 nonzeros
Variable types: 0 continuous, 11 integer (11 binary)
Found heuristic solution: objective 34.0000000

Root relaxation: objective 4.200000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

In [56]:
#deleting the normal row matrix

for i in range(len(xtrain_list)):
    for j in range(len(xtrain_list[i])):
        del xtrain_list[i][j][0:n]

for i in range(len(xtest_list)):
    for j in range(len(xtest_list[i])):     
        del xtest_list[i][j][0:n]

In [57]:
xtrain = np.array(xtrain_list)
ytrain = np.array(ytrain_list)

xtest = np.array(xtest_list)
ytest = np.array(ytest_list)

In [58]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(200, 5, 3)
(200, 5)
(100, 5, 3)
(100, 5)


In [59]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

def build_binary_model(xtrain, ytrain, xtest, ytest, nrlayers, nrnodes, rate, k):
    # This function builds the model with nrlayes layers and nrnodes nodes in each layer
    # nrlayers is a number and nrnodes is a vector specifying how many nodes each layer should contain
    model = keras.Sequential()
    numclasses = k

    # add the layers
    for i in range(nrlayers + 1):
        if (i == 0):
            # input layer
            model.add(keras.Input(shape=input_shape))
        else:
            # hidden layers
            nrnode = nrnodes[i - 1]
            model.add(keras.layers.Dense(nrnode, activation="sigmoid"))
            
    model.add(keras.layers.Dense(numclasses, activation="sigmoid"))

    # give summary of model
    model.summary()

    # train model
    batch_size = 30 # Based on SGD --> 1 gradient is calculated using 32 data points every time
    #so in each epoch, training_size/batch_size many gradient calculations and weight updates
    epochs = 10 # How many times to go through the data to complete the SGD

    opt = keras.optimizers.SGD(learning_rate=rate)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])
    
    model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs, validation_split=0.1,)
    score_train = model.evaluate(xtrain, ytrain, verbose=1)
    print(f"Train loss:, {score_train[0]}, Train accuracy:, {score_train[1]}")
    # get results
    score = model.evaluate(xtest, ytest, verbose=1)
    print(f"Test loss: {score[0]}, Test accuracy:, {score[1]}")
    return model

In [60]:
xtrain=np.reshape(xtrain,[200,15])
xtest=np.reshape(xtest,[100,15])

input_shape = (15,)
models1 = build_binary_model(xtrain, ytrain, xtest, ytest, 2, [50,25], 0.02, n)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 50)                800       
                                                                 
 dense_7 (Dense)             (None, 25)                1275      
                                                                 
 dense_8 (Dense)             (None, 5)                 130       
                                                                 
Total params: 2205 (8.61 KB)
Trainable params: 2205 (8.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10


6/6 [==============================] - 1s 54ms/step - loss: 0.6637 - accuracy: 0.1000 - val_loss: 0.6365 - val_accuracy: 0.1500
Epoch 2/10
6/6 [==============================] - 0s 14ms/step - loss: 0.6152 - accuracy: 0.1667 - val_loss: 0.5910 - val_accuracy: 0.1500
Epoch 3/10
6/6 [==============================] - 0s 11ms/step - loss: 0.5713 - accuracy: 0.1667 - val_loss: 0.5497 - val_accuracy: 0.3000
Epoch 4/10
6/6 [==============================] - 0s 11ms/step - loss: 0.5315 - accuracy: 0.1944 - val_loss: 0.5122 - val_accuracy: 0.3500
Epoch 5/10
6/6 [==============================] - 0s 15ms/step - loss: 0.4952 - accuracy: 0.2111 - val_loss: 0.4781 - val_accuracy: 0.3500
Epoch 6/10
6/6 [==============================] - 0s 10ms/step - loss: 0.4622 - accuracy: 0.2222 - val_loss: 0.4469 - val_accuracy: 0.3500
Epoch 7/10
6/6 [==============================] - 0s 12ms/step - loss: 0.4321 - accuracy: 0.2444 - val_loss: 0.4184 - val_accuracy: 0.3500
Epoch 8/10
6/6 [======================

In [61]:
# # full xtrain
# xtrain=np.reshape(xtrain,[200,88])
# xtest=np.reshape(xtest,[100,88])

# input_shape = (88,)
# models1 = build_binary_model(xtrain, ytrain, xtest, ytest, 2, [100,50], 0.02, n)

In [62]:
test1 = models1.predict(xtest)
print(models1.predict(xtest))

4/4 [==============================] - 0s 135us/step
[[0.711975   0.30228856 0.33623743 0.23732102 0.7541337 ]
 [0.68962604 0.31313646 0.29447278 0.28470272 0.71520996]
 [0.69252926 0.3078546  0.29804876 0.27715638 0.7277165 ]
 [0.70891756 0.30480036 0.3236704  0.23825176 0.7737471 ]
 [0.7033955  0.31144574 0.28402442 0.28167802 0.7120549 ]
 [0.7079074  0.29663107 0.30850568 0.23156966 0.77166915]
 [0.70938164 0.31253693 0.34681943 0.22596851 0.7660789 ]
 [0.7175501  0.3051325  0.2695915  0.29437923 0.6814284 ]
 [0.7069013  0.3013674  0.26976424 0.2871585  0.68600255]
 [0.69252926 0.3078546  0.29804876 0.27715638 0.7277165 ]
 [0.706662   0.28767702 0.2996705  0.24006419 0.7629646 ]
 [0.707874   0.30334216 0.277708   0.29358584 0.6942445 ]
 [0.711975   0.30228856 0.33623743 0.23732102 0.7541337 ]
 [0.70891756 0.30480036 0.3236704  0.23825176 0.7737471 ]
 [0.69252926 0.3078546  0.29804876 0.27715638 0.7277165 ]
 [0.71212894 0.3098304  0.34120992 0.22985066 0.76480305]
 [0.7175501  0.3051

In [63]:
ytest

array([[0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1,